<a href="https://colab.research.google.com/github/saied-ahemd/TextClassificationWithBidirectional/blob/main/sentementClassificarionDeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dataset, info = tfds.load('imdb_reviews', with_info=True,as_supervised=True)
# now let's get the train data and the test data
train_set, test_set = dataset['train'], dataset['test']

In [ ]:
for example, label in train_set.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

In [ ]:
# now we will shuffle our data
BUFFER_SIZE = 10000
BATCH_SIZE = 64
train_set = train_set.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
test_set = test_set.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# now we will create the encoder 
VOCAB_SIZE=1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_set.map(lambda text, label: text))

In [ ]:
# now i we will create the model
encoder_len = len(encoder.get_vocabulary())
model = tf.keras.Sequential([
                             encoder,
                             tf.keras.layers.Embedding(encoder_len,64,mask_zero=True),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
                             tf.keras.layers.Dense(64,activation='relu'),
                             tf.keras.layers.Dense(1,activation='sigmoid')
])


In [ ]:
# let's compile our model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])

In [ ]:
his = model.fit(train_set,epochs=10,validation_data=test_set,validation_steps=30)

In [ ]:
# now we can eval our model
test_loss, test_acc = model.evaluate(test_set)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

In [ ]:
# now let's make a prediction 
# predict on a sample text without padding.

sample_text = ('this movie was very good')
predictions = model.predict(np.array([sample_text]))
print(predictions)

In [ ]:
# now we can plot the loss function and the acc 
def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [ ]:
plt.figure(figsize=(16,6))
plt.subplot(1,2,1)
plot_graphs(his, 'acc')
plt.subplot(1,2,2)
plot_graphs(his, 'loss')